# Strategy Simulation (fast version)

In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
import financial.data as fd
import financial.portfolio as fp
import financial.strategies.simulation as fss

import os

In [39]:
from financial.io.cache import NoUpdateStrategy
from financial.io.file.cache import FileCache

print(os.environ["DATA"])
print(os.environ["CACHE"])
ds = fd.CachedDataStore(path=os.environ["DATA"], cache=FileCache(cache_path=os.environ["CACHE"]+"/", update_strategy=NoUpdateStrategy()))

print(ds)

/home/manidmt/Universidad/TFG/OTRI/data
/home/manidmt/Universidad/TFG/OTRI/cache
CachedDataStore with 946 data sources [cache stats: {'size': 0, 'hit': 0, 'miss': 0, 'write': 0, 'read': 0, 'update': 0}]


# Signals

In [40]:
import financial.strategies.technical.indicator as st

# Momentum
    
momentum1m = st.Momentum()
momentum1m.set_parameters({'delay': 20})

momentum3m = st.Momentum()
momentum3m.set_parameters({'delay': 60})

momentum4m = st.Momentum()
momentum4m.set_parameters({'delay': 84})

momentum6m = st.Momentum()
momentum6m.set_parameters({'delay': 126})

momentum8m = st.Momentum()
momentum8m.set_parameters({'delay': 168})

momentum9m = st.Momentum()
momentum9m.set_parameters({'delay': 189})
       
momentum1y = st.Momentum()
momentum1y.set_parameters({'delay': 252})
    
# Aggregation

indicator_ferrer = st.Mean()
indicator_ferrer.set_parameters({'components': [momentum4m.to_dict(), momentum6m.to_dict(), momentum8m.to_dict()]})
print(indicator_ferrer.to_dict())    
    
indicator_requejo = st.Mean() # st.Sum() # Average instead of sum to make it comparable
indicator_requejo.set_parameters({'components': [momentum1m.to_dict(), momentum3m.to_dict(), momentum6m.to_dict(), momentum9m.to_dict(), momentum1y.to_dict()]})
print(indicator_requejo.to_dict())  


{'type': 'financial.strategies.technical.indicator.Mean', 'components': [{'type': 'financial.strategies.technical.indicator.Momentum', 'delay': 84}, {'type': 'financial.strategies.technical.indicator.Momentum', 'delay': 126}, {'type': 'financial.strategies.technical.indicator.Momentum', 'delay': 168}]}
{'type': 'financial.strategies.technical.indicator.Mean', 'components': [{'type': 'financial.strategies.technical.indicator.Momentum', 'delay': 20}, {'type': 'financial.strategies.technical.indicator.Momentum', 'delay': 60}, {'type': 'financial.strategies.technical.indicator.Momentum', 'delay': 126}, {'type': 'financial.strategies.technical.indicator.Momentum', 'delay': 189}, {'type': 'financial.strategies.technical.indicator.Momentum', 'delay': 252}]}


In [41]:
from financial.momentum.exponentialIndicator import ExponentialRegressionIndicator

indicator_exponential = ExponentialRegressionIndicator()

# Strategies

In [42]:
import financial.strategies.allocation as fsa
import financial.strategies.filter as fsf
import financial.strategies.rank as fsr

# Asset Universe

universe_ferrer = ['XLI', 'XLF', 'XLV', 'XLK', 'XLC', 'XLU', 'XLY', 'XLB', 'XLE', 'XLP',
                    'TLT', 'TIP', 'IEF', 'SHY', 'AGG', 'VNQ', 'GLD', 'DBC'] # IAU excluded
    
universe_requejo = ['XLE', 'XLF', 'XLU', 'XLI', 'XLK', 'XLV', 'XLY', 'XLP', 'XLB', 'XOP',
                    'XHB', 'XME', 'XRT', 'XTL', 'TLT', 'DBC']
    
universe_ing = ['XLI', 'XLF', 'XLV', 'XLK', 'XLY', 'XLB', 'XLP', 
                'SPY', 'DJE', 'NASD', 
                'IOGP', 'IQQH', 'RBOT', 'HEAL', 
                'PHAU', 'LYTR', 'GLRE']

universe_ing_eur = ['ZPDM', 'ZPDI', 'ZPDS', 'ZPDH', 'ZPDT', 'ZPDD', 'ZPDF',
                   'SPY5', 'DJE', 'UST', 'LYTR', 'IQQH'] # , 'IOGP', 'GLRE', 'PHAU', 'HEAL', 'RBOT']

from typing import List

def universe_report(tickers: List[str]):
    for ticker in tickers:
        metadata = ds.get_metadata(ticker)
        #print(metadata)
        print(f"{ticker:4} {metadata['currency']} {metadata['since']} {metadata['isin']} {metadata['description']}")

print("HUGO FERRER'S UNIVERSE")
universe_report(universe_ferrer)

print("DANIEL REQUEJO'S UNIVERSE")
universe_report(universe_requejo)

print("ING'S UNIVERSE")
universe_report(universe_ing)

print("ING'S UNIVERSE [EUR]")
universe_report(universe_ing_eur)


# Asset ranking strategies

ferrer_refuge = None # refuge
ferrer_filter = fsf.CompositeAssetFilter( [fsf.TopKAssetFilter(k=10), fsf.MinimumValueAssetFilter(threshold=0.0)] )
ferrer_allocation = fsa.FixedWeightAllocation(0.10)
ferrer_strategy = fsr.AssetRankingStrategy("HUGO FERRER'S STRATEGY", universe_ferrer,indicator_ferrer, ferrer_filter, ferrer_allocation, ferrer_refuge)
print(ferrer_strategy)

requejo_filter = fsf.TopKAssetFilter(k=3)
requejo_allocation = fsa.EqualWeightAllocation()
requejo_strategy = fsr.AssetRankingStrategy("DANIEL REQUEJO'S STRATEGY", universe_requejo, indicator_requejo, requejo_filter, requejo_allocation)    
print(requejo_strategy)

ferrer_strategy_ing = fsr.AssetRankingStrategy("ING @ Ferrer's strategy", universe_ing_eur, indicator_ferrer, ferrer_filter, ferrer_allocation)
print(ferrer_strategy_ing)

requejo_strategy_ing = fsr.AssetRankingStrategy("ING @ Requejo's strategy", universe_ing_eur, indicator_requejo, requejo_filter, requejo_allocation)
print(requejo_strategy_ing)

exp_filter = fsf.TopKAssetFilter(k=10)
exp_allocation = fsa.EqualWeightAllocation()
exp_strategy = fsr.AssetRankingStrategy("Exponential Strategy", universe_ing_eur, indicator_exponential, exp_filter, exp_allocation)
print(exp_strategy)


HUGO FERRER'S UNIVERSE
XLI  USD 02/01/2001 US81369Y7040 Industrial SPDR Select Sector ETF
XLF  USD 30/03/2001 US81369Y6059 Financial SPDR Select Sector ETF
XLV  USD 02/01/2001 US81369Y2090 Health Care SPDR Select Sector ETF
XLK  USD 02/01/2001 US81369Y8030 Technology SPDR Select Sector ETF
XLC  USD 20/06/2018 US81369Y8527 Communication Services SPDR Select Sector ETF
XLU  USD 27/11/2017 US81369Y8865 Utilities SPDR Select Sector ETF
XLY  USD 02/01/2001 US81369Y4070 Consumer Discretionary SPDR Select Sector ETF
XLB  USD 02/01/2001 US81369Y1001 Materials SPDR Select Sector ETF
XLE  USD 30/03/2001 US81369Y5069 Energy SPDR Select Sector ETF
XLP  USD 02/01/2001 US81369Y3080 Consumer Staples SPDR Select Sector ETF
TLT  USD 30/07/2002 US4642874329 US Bond 20+yr iShares ETF
TIP  USD 05/12/2003 US4642871762 US TIPS Bond iShares ETF
IEF  USD 07/07/2015 US4642874402 US Bond 7-10yr iShares ETF NASDAQ
SHY  USD 07/07/2015 US4642874576 US Bond 1-3yr iShares ETF NASDAQ
AGG  USD 29/09/2003 US4642872265 

# Porfolio performance

In [43]:
portfolio = fp.StockPortfolio("Stocks")
portfolio.add('$$$', '2001-01-01', 1000000, 1000000)

print(portfolio)

for ticker in portfolio.lots:
    print(portfolio.lots[ticker])
    
print(f"- Cost:  ${portfolio.cost():10.2f}")
print(f"- Value: ${portfolio.value(ds):10.2f}")
print(f"- Gain:  ${portfolio.gain(ds):>10.2f}  {100*portfolio.gain(ds)/portfolio.cost():>6.2f}%")
print()

fss.SimulationReport.value_report(ds,portfolio,start_year=2000)
print()
fss.SimulationReport.performance_report(ds,portfolio,start_year=2000)
fss.SimulationReport.performance_report_stocks(ds,portfolio,start_year=2000)


Stocks: ['$$$']
$$$: [{'index': '2001-01-01', 'stocks': 1000000, 'cost': 1000000}]
- Cost:  $1000000.00
- Value: $1000000.00
- Gain:  $      0.00    0.00%

           YEAR        Jan        Feb        Mar        Apr        May        Jun        Jul        Aug        Sep        Oct        Nov        Dec
  2000:       0.00                                                                                                
  2001: 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00
  2002: 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00
  2003: 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00
  2004: 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00 1000000.00

In [44]:
import financial.portfolios.statistics as fps

assets = {'^GSPC': 1.0}
target = fp.WeightedPortfolio.from_assets("Target", assets)
print(target)

print("Daily returns")
benchmark_returns = target.returns(ds,'2015-12-31','2023-12-31')
print(benchmark_returns)
print("Monthly returns")
benchmark_monthly_returns = fss.SimulationUtilities.monthly_returns(ds,target,2016,2023)
print(benchmark_monthly_returns)
benchmark_cumulative_returns = fps.CumulativeReturn().get_series(benchmark_monthly_returns)
print("Cumulative returns")
print(benchmark_cumulative_returns)


fss.SimulationReport.performance_report(ds, target, 2001)

print("\n Cumulative return")
fss.SimulationReport.cumulative_report(ds, target, 2001)


Target:
      ticker  weight  cost  isin description
^GSPC  ^GSPC     1.0  None  None        None
Daily returns
2016-01-04   -0.015304
2016-01-05    0.002012
2016-01-06   -0.013115
2016-01-07   -0.023700
2016-01-08   -0.010838
                ...   
2023-12-22    0.001660
2023-12-26    0.004232
2023-12-27    0.001430
2023-12-28    0.000370
2023-12-29   -0.002826
Length: 2012, dtype: float64
Monthly returns
2016-01-31   -0.050735
2016-02-29   -0.004128
2016-03-31    0.065991
2016-04-30    0.002699
2016-05-31    0.015325
                ...   
2023-08-31   -0.017716
2023-09-30   -0.048719
2023-10-31   -0.021980
2023-11-30    0.089179
2023-12-31    0.044229
Name: return, Length: 96, dtype: float64
Cumulative returns
2016-01-31   -0.050735
2016-02-29   -0.054654
2016-03-31    0.007730
2016-04-30    0.010450
2016-05-31    0.025935
                ...   
2023-08-31    1.205378
2023-09-30    1.097933
2023-10-31    1.051821
2023-11-30    1.234801
2023-12-31    1.333645
Name: return, Length: 96

# Strategy simulation 

In [53]:
%%time
import financial.strategies.rebalance as fsrebalance

strategy = exp_strategy #ferrer_strategy_ing #ferrer_strategy
benchmark = fp.BenchmarkPortfolio("^GSPC")
market = ds.get_data("^GSPC")
rebalancing = fsrebalance.MonthlyRebalancingSchedule(+2,market) # +1 first day, -1 last day

print("STRATEGY")
print(strategy)
print("BENCHMARK")
print(benchmark)
print("REBALANCING")
print(rebalancing)
        
simulation = fss.FastStockStrategySimulation(ds, strategy, rebalancing, market)
#simulation.verbose = True  # verbose 30.4s vs. 15.4s without trading details
simulation.simulate(start_year=2016, end_year=2023)


STRATEGY
Exponential Strategy
- 12 asset universe: ['ZPDM', 'ZPDI', 'ZPDS', 'ZPDH', 'ZPDT', 'ZPDD', 'ZPDF', 'SPY5', 'DJE', 'UST', 'LYTR', 'IQQH']
- Indicator: {'type': 'financial.momentum.exponentialIndicator.ExponentialRegressionIndicator', 'lookahead': 20, 'horizon': 90, 'model': 'exponential'}
BENCHMARK
^GSPC(^GSPC)
REBALANCING
Rebalancing schedule: Monthly rebalancing on 2
SIMULATING 2016...
+ Rebalance @ 2015-12-01
{'ZPDM': 0.1, 'ZPDI': 0.1, 'ZPDS': 0.1, 'ZPDH': 0.1, 'ZPDT': 0.1, 'ZPDD': 0.1, 'ZPDF': 0.1, 'SPY5': 0.1, 'DJE': 0.1, 'UST': 0.1}
+ Trade @ 2015-12-02
- Changes: {'ZPDI': 5178, 'ZPDF': 5241, '$$$': -999663.864032149, 'ZPDH': 5454, 'SPY5': 502, 'DJE': 584, 'ZPDT': 4926, 'UST': 5705, 'ZPDS': 5085, 'ZPDM': 5523, 'ZPDD': 4995}
- Rotation: 99.97%
+ Rebalance @ 2015-12-31
{'ZPDM': 0.1, 'ZPDI': 0.1, 'ZPDS': 0.1, 'ZPDH': 0.1, 'ZPDT': 0.1, 'ZPDD': 0.1, 'ZPDF': 0.1, 'SPY5': 0.1, 'DJE': 0.1, 'UST': 0.1}
+ Trade @ 2016-01-04


/home/manidmt/Universidad/TFG/OTRI/src/financial/strategies/trade.py:109: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.ds.get_data(ticker, None, self.trading_date)[-1]
/home/manidmt/Universidad/TFG/OTRI/src/financial/strategies/trade.py:107: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.ds.get_data(ticker_price, None, self.trading_date)[-1]
/home/manidmt/Universidad/TFG/OTRI/src/financial/strategies/trade.py:109: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). T

- Changes: {'ZPDI': 12, 'ZPDF': 54, '$$$': -160.1048664502968, 'ZPDH': -127, 'SPY5': 3, 'DJE': 8, 'ZPDT': 49, 'UST': -28, 'ZPDS': -237, 'ZPDM': 130, 'ZPDD': 74}
- Rotation: 0.79%
+ Rebalance @ 2016-02-01
{'ZPDM': 0.1, 'ZPDI': 0.1, 'ZPDS': 0.1, 'ZPDH': 0.1, 'ZPDT': 0.1, 'ZPDD': 0.1, 'ZPDF': 0.1, 'SPY5': 0.1, 'DJE': 0.1, 'UST': 0.1}
+ Trade @ 2016-02-02
- Changes: {'ZPDI': 41, 'ZPDF': 232, '$$$': -206.87421684587008, 'ZPDH': 84, 'SPY5': -4, 'DJE': -2, 'ZPDT': -162, 'UST': 120, 'ZPDS': -315, 'ZPDM': 268, 'ZPDD': -72}


/home/manidmt/Universidad/TFG/OTRI/src/financial/strategies/trade.py:109: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.ds.get_data(ticker, None, self.trading_date)[-1]
/home/manidmt/Universidad/TFG/OTRI/src/financial/strategies/trade.py:107: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.ds.get_data(ticker_price, None, self.trading_date)[-1]


- Rotation: 1.31%
+ Rebalance @ 2016-03-01
{'ZPDM': 0.1, 'ZPDI': 0.1, 'ZPDS': 0.1, 'ZPDH': 0.1, 'ZPDT': 0.1, 'ZPDD': 0.1, 'ZPDF': 0.1, 'SPY5': 0.1, 'DJE': 0.1, 'UST': 0.1}
+ Trade @ 2016-03-02


...

- Changes: {'ZPDI': -25, 'ZPDF': -34, '$$$': -532.5900033568164, 'ZPDH': -5, 'DJE': -3, 'ZPDT': 5, 'UST': 2, 'ZPDS': 3, 'ZPDM': 6, 'ZPDD': 70}
- Rotation: 0.38%
+ Rebalance @ 2023-09-01
{'ZPDM': 0.1, 'ZPDI': 0.1, 'ZPDS': 0.1, 'ZPDH': 0.1, 'ZPDT': 0.1, 'ZPDD': 0.1, 'ZPDF': 0.1, 'SPY5': 0.1, 'DJE': 0.1, 'UST': 0.1}
+ Trade @ 2023-09-05
- Changes: {'ZPDI': 30, 'ZPDF': 2, '$$$': 85.21004806518503, 'ZPDH': -12, 'SPY5': -2, 'ZPDT': -28, 'UST': -35, 'ZPDS': 133, 'ZPDM': 30, 'ZPDD': -27}


/home/manidmt/Universidad/TFG/OTRI/src/financial/strategies/trade.py:109: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.ds.get_data(ticker, None, self.trading_date)[-1]
/home/manidmt/Universidad/TFG/OTRI/src/financial/strategies/trade.py:107: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.ds.get_data(ticker_price, None, self.trading_date)[-1]
/home/manidmt/Universidad/TFG/OTRI/src/financial/strategies/trade.py:109: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). T

- Rotation: 0.65%
+ Rebalance @ 2023-09-29
{'ZPDM': 0.1, 'ZPDI': 0.1, 'ZPDS': 0.1, 'ZPDH': 0.1, 'ZPDT': 0.1, 'ZPDD': 0.1, 'ZPDF': 0.1, 'SPY5': 0.1, 'DJE': 0.1, 'UST': 0.1}
+ Trade @ 2023-10-02
- Changes: {'ZPDI': 31, 'ZPDF': -14, '$$$': 65.71999313360313, 'ZPDH': -39, 'DJE': -2, 'ZPDT': 18, 'UST': -9, 'ZPDS': -12, 'ZPDM': 34, 'ZPDD': -12}
- Rotation: 0.39%
+ Rebalance @ 2023-11-01
{'ZPDM': 0.1, 'ZPDI': 0.1, 'ZPDS': 0.1, 'ZPDH': 0.1, 'ZPDT': 0.1, 'ZPDD': 0.1, 'ZPDF': 0.1, 'SPY5': 0.1, 'DJE': 0.1, 'UST': 0.1}
+ Trade @ 2023-11-02
- Changes: {'ZPDI': 15, 'ZPDF': -23, '$$$': -479.71499465946357, 'ZPDH': 42, 'DJE': -3, 'ZPDT': -24, 'UST': -5, 'ZPDS': -16, 'ZPDM': 12, 'ZPDD': 59}
- Rotation: 0.49%
+ Rebalance @ 2023-12-01
{'ZPDM': 0.1, 'ZPDI': 0.1, 'ZPDS': 0.1, 'ZPDH': 0.1, 'ZPDT': 0.1, 'ZPDD': 0.1, 'ZPDF': 0.1, 'SPY5': 0.1, 'DJE': 0.1, 'UST': 0.1}
+ Trade @ 2023-12-04


/home/manidmt/Universidad/TFG/OTRI/src/financial/strategies/trade.py:109: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.ds.get_data(ticker, None, self.trading_date)[-1]
/home/manidmt/Universidad/TFG/OTRI/src/financial/strategies/trade.py:107: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.ds.get_data(ticker_price, None, self.trading_date)[-1]
/home/manidmt/Universidad/TFG/OTRI/src/financial/strategies/trade.py:109: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). T

- Changes: {'ZPDI': -30, 'ZPDF': -52, '$$$': 164.9549862671713, 'ZPDH': 54, 'SPY5': 1, 'DJE': -2, 'ZPDT': -10, 'UST': 9, 'ZPDS': 94, 'ZPDD': -30}
- Rotation: 0.57%
+ Rebalance @ 2023-12-29
{'ZPDM': 0.1, 'ZPDI': 0.1, 'ZPDS': 0.1, 'ZPDH': 0.1, 'ZPDT': 0.1, 'ZPDD': 0.1, 'ZPDF': 0.1, 'SPY5': 0.1, 'DJE': 0.1, 'UST': 0.1}
FINAL PORTFOLIO @ 2023
Stocks: ['$$$', 'ZPDI', 'ZPDF', 'ZPDH', 'SPY5', 'DJE', 'ZPDT', 'UST', 'ZPDS', 'ZPDM', 'ZPDD']
Portfolio value: 1106687.70
Portfolio assets: {'$$$': -229.33075988673352, 'ZPDI': 111442.28, 'ZPDF': 110719.22, 'ZPDH': 109132.79999999999, 'SPY5': 110256.9, 'DJE': 108699.25, 'ZPDT': 112158.54000000001, 'UST': 111986.81803894043, 'ZPDS': 107157.96, 'ZPDM': 110338.2, 'ZPDD': 111532.2}
Portfolio stocks: {'$$$': -229.33075988673352, 'ZPDI': 2549, 'ZPDF': 2818, 'ZPDH': 2940, 'SPY5': 255, 'DJE': 319, 'ZPDT': 1229, 'UST': 1836, 'ZPDS': 3276, 'ZPDM': 2940, 'ZPDD': 2353}
Portfolio weights: {'$$$': -0.00020787874647089577, 'ZPDI': 0.10101776788120576, 'ZPDF': 0.1003

In [46]:
simulation.portfolio_target_report()


2015-12-01: {'ZPDM': 0.1, 'ZPDI': 0.1, 'ZPDS': 0.1, 'ZPDH': 0.1, 'ZPDT': 0.1, 'ZPDD': 0.1, 'ZPDF': 0.1, 'SPY5': 0.1, 'DJE': 0.1, 'UST': 0.1}
2015-12-31: {'ZPDM': 0.1, 'ZPDI': 0.1, 'ZPDS': 0.1, 'ZPDH': 0.1, 'ZPDT': 0.1, 'ZPDD': 0.1, 'ZPDF': 0.1, 'SPY5': 0.1, 'DJE': 0.1, 'UST': 0.1}
2016-02-01: {'ZPDM': 0.1, 'ZPDI': 0.1, 'ZPDS': 0.1, 'ZPDH': 0.1, 'ZPDT': 0.1, 'ZPDD': 0.1, 'ZPDF': 0.1, 'SPY5': 0.1, 'DJE': 0.1, 'UST': 0.1}
2016-03-01: {'ZPDM': 0.1, 'ZPDI': 0.1, 'ZPDS': 0.1, 'ZPDH': 0.1, 'ZPDT': 0.1, 'ZPDD': 0.1, 'ZPDF': 0.1, 'SPY5': 0.1, 'DJE': 0.1, 'UST': 0.1}
2016-04-01: {'ZPDM': 0.1, 'ZPDI': 0.1, 'ZPDS': 0.1, 'ZPDH': 0.1, 'ZPDT': 0.1, 'ZPDD': 0.1, 'ZPDF': 0.1, 'SPY5': 0.1, 'DJE': 0.1, 'UST': 0.1}
2016-04-29: {'ZPDM': 0.1, 'ZPDI': 0.1, 'ZPDS': 0.1, 'ZPDH': 0.1, 'ZPDT': 0.1, 'ZPDD': 0.1, 'ZPDF': 0.1, 'SPY5': 0.1, 'DJE': 0.1, 'UST': 0.1}
2016-06-01: {'ZPDM': 0.1, 'ZPDI': 0.1, 'ZPDS': 0.1, 'ZPDH': 0.1, 'ZPDT': 0.1, 'ZPDD': 0.1, 'ZPDF': 0.1, 'SPY5': 0.1, 'DJE': 0.1, 'UST': 0.1}
2016-07-01: {

In [47]:
simulation.simulation_trading_report()

2015-12-02: 99.97% {'ZPDI': 5178, 'ZPDF': 5241, '$$$': -999663.864032149, 'ZPDH': 5454, 'SPY5': 502, 'DJE': 584, 'ZPDT': 4926, 'UST': 5705, 'ZPDS': 5085, 'ZPDM': 5523, 'ZPDD': 4995}
2016-01-04: 0.79% {'ZPDI': 12, 'ZPDF': 54, '$$$': -160.1048664502968, 'ZPDH': -127, 'SPY5': 3, 'DJE': 8, 'ZPDT': 49, 'UST': -28, 'ZPDS': -237, 'ZPDM': 130, 'ZPDD': 74}
2016-02-02: 1.31% {'ZPDI': 41, 'ZPDF': 232, '$$$': -206.87421684587008, 'ZPDH': 84, 'SPY5': -4, 'DJE': -2, 'ZPDT': -162, 'UST': 120, 'ZPDS': -315, 'ZPDM': 268, 'ZPDD': -72}
2016-03-02: 0.96% {'ZPDI': -182, 'ZPDF': 37, '$$$': 211.0557345509401, 'ZPDH': 100, 'SPY5': 4, 'ZPDT': 102, 'UST': 39, 'ZPDS': 111, 'ZPDM': -334, 'ZPDD': 48}


/home/manidmt/Universidad/TFG/OTRI/src/financial/strategies/trade.py:109: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.ds.get_data(ticker, None, self.trading_date)[-1]
/home/manidmt/Universidad/TFG/OTRI/src/financial/strategies/trade.py:107: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.ds.get_data(ticker_price, None, self.trading_date)[-1]


2016-04-04: 0.36% {'ZPDI': -4, 'ZPDF': 60, '$$$': -152.1071060298982, 'ZPDH': 76, 'SPY5': 1, 'DJE': -4, 'ZPDT': -87, 'UST': 27, 'ZPDM': -47, 'ZPDD': 25}
2016-05-02: 1.09% {'ZPDI': -77, 'ZPDF': -191, '$$$': 32.782773738969354, 'ZPDH': -1, 'DJE': -1, 'ZPDT': 298, 'UST': 197, 'ZPDS': 87, 'ZPDM': -247, 'ZPDD': -51}
2016-06-02: 0.76% {'ZPDI': 105, 'ZPDF': 15, '$$$': 27.462328063936866, 'ZPDH': -127, 'DJE': 7, 'ZPDT': -143, 'UST': -142, 'ZPDS': 24, 'ZPDM': 82, 'ZPDD': 103}
2016-07-05: 0.91% {'ZPDI': -46, 'ZPDF': 250, '$$$': -307.5853328665432, 'ZPDH': -51, 'DJE': -6, 'ZPDT': 18, 'UST': 121, 'ZPDS': -265, 'ZPDM': 94, 'ZPDD': 33}
2016-08-02: 0.84% {'ZPDI': 25, 'ZPDF': -52, '$$$': 453.25889009231594, 'ZPDH': -25, 'SPY5': 1, 'DJE': 12, 'ZPDT': -144, 'UST': -124, 'ZPDS': 236, 'ZPDM': -103, 'ZPDD': 11}
2016-09-02: 0.65% {'ZPDI': -51, 'ZPDF': -176, '$$$': -233.70983747269048, 'ZPDH': 255, 'ZPDT': -57, 'UST': -21, 'ZPDS': 47, 'ZPDM': -24, 'ZPDD': 50}
2016-10-03: 0.54% {'ZPDI': -25, 'ZPDF': 97, '$$$'

In [48]:
rotation = simulation.monthly_rotation()
print(f"Average monthly rotation: {100*rotation:.2f}%")

Average monthly rotation: 9.11%


In [49]:
year_returns = simulation.year_returns()
#print(year_returns)

print("\nSTRATEGY RETURNS")
for year in year_returns:
    print(f"{year}: {100*year_returns[year]:6.2f}%")
    

cumulative = simulation.cumulative_returns(year_returns)

print("\nCUMULATIVE RETURNS")
for year in cumulative:
    print(f"{year}: {100*cumulative[year]:6.2f}%")



STRATEGY RETURNS
2016:  14.20%
2017:   9.05%
2018:  -1.51%
2019:  32.36%
2020:   7.72%
2021:  38.35%
2022: -11.42%
2023:  19.05%

CUMULATIVE RETURNS
2016:  14.20%
2017:  24.53%
2018:  22.65%
2019:  62.34%
2020:  74.87%
2021: 141.93%
2022: 114.31%
2023: 155.13%


In [50]:
simulation.simulation_year_report()

2016
- Cost:  $1000000.00
- Value: $1097351.91
- Gain:  $  97351.91    9.74%
2017
- Cost:  $1000000.00
- Value: $1130226.19
- Gain:  $ 130226.19   13.02%
2018
- Cost:  $1000000.00
- Value: $ 998173.03
- Gain:  $  -1826.97   -0.18%
2019
- Cost:  $1000000.00
- Value: $1323644.44
- Gain:  $ 323644.44   32.36%
2020
- Cost:  $1000000.00
- Value: $1077221.04
- Gain:  $  77221.04    7.72%
2021
- Cost:  $1000000.00
- Value: $1388934.97
- Gain:  $ 388934.97   38.89%
2022
- Cost:  $1000000.00
- Value: $ 932594.43
- Gain:  $ -67405.57   -6.74%
2023
- Cost:  $1000000.00
- Value: $1106687.70
- Gain:  $ 106687.70   10.67%


In [51]:
%%time

year = 2016
monthly_returns = simulation.monthly_returns(year)
#print(monthly_returns)
cumulative_returns = simulation.cumulative_returns(monthly_returns)
year_return = cumulative_returns[12]

print("Monthly returns")
returns_str = [f"{100*month_return:>6.2f}%" for month_return in monthly_returns.values()]
print(f"  {year}:           {100*year_return:>6.2f}% " + " ".join(returns_str))

print("Monthly cumulative returns")
returns_str = [f"{100*month_return:>6.2f}%" for month_return in cumulative_returns.values()]
print(f"  {year}:           {100*year_return:>6.2f}% " + " ".join(returns_str))

print()
print("SERIES")
print("Monthly returns")
monthly_returns = simulation.returns()
print(monthly_returns)
cumulative_returns = fps.CumulativeReturn().get_series(monthly_returns)
print("Cumulative returns")
print(cumulative_returns)

Monthly returns
  2016:            14.20%  -7.36%   2.56%   0.59%  -0.76%   4.89%  -0.49%   4.01%   0.17%  -0.85%   0.98%   8.11%   2.30%
Monthly cumulative returns
  2016:            14.20%  -7.36%  -4.99%  -4.43%  -5.16%  -0.52%  -1.01%   2.96%   3.13%   2.25%   3.26%  11.63%  14.20%

SERIES
Monthly returns


AttributeError: 'Series' object has no attribute 'append'

In [52]:
difference = monthly_returns - benchmark_monthly_returns
print(difference)

print("Better than benchmark")
print(sum(difference>0))
print(difference[difference>0])
print(sum(difference[difference>0]))

print("Worse than benchmark")
print(sum(difference<0))
print(difference[difference<0])
print(sum(difference[difference<0]))

TypeError: unsupported operand type(s) for -: 'dict' and 'float'

In [ ]:
drawdown = fps.MaximumDrawdown().get_series(monthly_returns)
print(drawdown)
print(min(drawdown))

benchmark_drawdown = fps.MaximumDrawdown().get_series(benchmark_returns) # benchmark_monthly_returns)
print(benchmark_drawdown)
print(min(benchmark_drawdown))

In [ ]:
%%time

simulation.simulation_performance_report()

In [ ]:
%%time

print("MONTHLY RETURNS")
fss.SimulationReport.simulation_report(monthly_returns, fss.accumulate_return)
print("CUMULATIVE RETURNS")
fss.SimulationReport.simulation_report(cumulative_returns, fss.accumulate_last)
print("SIMULATION DRAWDOWN")
fss.SimulationReport.simulation_report(drawdown, fss.accumulate_min)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import style

# Adjusting the size of matplotlib
import matplotlib as mpl

mpl.rc('figure', figsize=(16, 9))
mpl.__version__

# Adjusting the style of matplotlib
style.use('ggplot')

benchmark = 100*benchmark_cumulative_returns
#benchmark = 100*fps.MaximumDrawdown().get_series(benchmark_monthly_returns)

data = 100*cumulative_returns
#data = 100*drawdown

#monthly_returns.plot(label=ticker)
data.plot()
benchmark.plot()
print("STRATEGY vs. BENCHMARK")